In [1]:
!pip install allensdk

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache
import allensdk

c:\myprograms\Anaconda\envs\behavis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Confirming your allensdk version
print(f"Your allensdk version is: {allensdk.__version__}")

Your allensdk version is: 2.16.2


In [4]:
# Update this to a valid directory in your filesystem. This is where the data will be stored.
output_dir = "D:\Visual Behavior Neuropixels AI"
DOWNLOAD_COMPLETE_DATASET = False

In [5]:
output_dir = Path(output_dir)
cache = VisualBehaviorNeuropixelsProjectCache.from_s3_cache(cache_dir=output_dir)
# Load latest manifest
cache.load_latest_manifest()

ecephys_sessions.csv: 100%|██████████| 64.4k/64.4k [00:00<00:00, 224kMB/s] 
behavior_sessions.csv: 100%|██████████| 562k/562k [00:00<00:00, 785kMB/s]  
units.csv: 100%|██████████| 134M/134M [01:16<00:00, 1.76MMB/s]    
probes.csv: 100%|██████████| 130k/130k [00:00<00:00, 194kMB/s]  
channels.csv: 100%|██████████| 27.9M/27.9M [00:20<00:00, 1.37MMB/s]  


In [6]:
ecephys_sessions = cache.get_ecephys_session_table()
print(f"Total number of ecephys sessions: {len(ecephys_sessions)}")
behavior_sessions = cache.get_behavior_session_table()
print(f"Total number of behavior sessions: {len(behavior_sessions)}")
units = cache.get_unit_table()
print(f"Total number of units: {len(units)}")
probes = cache.get_probe_table()
print(f"Total number of probes: {len(probes)}")
channels = cache.get_channel_table()
print(f"Total number of channels: {len(channels)}")

Total number of ecephys sessions: 103
Total number of behavior sessions: 3424
Total number of units: 319013
Total number of probes: 905
Total number of channels: 347520


In [33]:
behavior_sessions.columns

Index(['equipment_name', 'genotype', 'mouse_id', 'sex', 'age_in_days',
       'session_number', 'prior_exposures_to_session_type',
       'prior_exposures_to_image_set', 'prior_exposures_to_omissions',
       'ecephys_session_id', 'date_of_acquisition', 'session_type',
       'image_set', 'file_id'],
      dtype='object')

In [32]:
behavior_sessions['session_type'].value_counts()

TRAINING_5_images_G_handoff_ready_5uL_reward        818
HABITUATION_5_images_G_handoff_ready_3uL_reward     436
TRAINING_5_images_G_epilogue_5uL_reward             419
TRAINING_1_gratings_10uL_reward                     311
TRAINING_2_gratings_flashed_10uL_reward             216
TRAINING_3_images_G_10uL_reward                     213
HABITUATION_5_images_G_handoff_ready_5uL_reward     177
TRAINING_5_images_G_handoff_lapsed_5uL_reward       159
TRAINING_4_images_G_training_7uL_reward             131
TRAINING_5_images_H_handoff_ready_5uL_reward         98
TRAINING_0_gratings_autorewards_15min_0uL_reward     81
EPHYS_1_images_H_3uL_reward                          77
EPHYS_1_images_G_3uL_reward                          77
HABITUATION_5_images_H_handoff_ready_3uL_reward      55
TRAINING_5_images_H_handoff_lapsed_5uL_reward        36
TRAINING_3_images_H_10uL_reward                      30
TRAINING_5_images_H_epilogue_5uL_reward              29
TRAINING_4_images_H_training_7uL_reward         

In [7]:
# mask = behavior_sessions['mouse_id'] == 544835
mask = behavior_sessions['session_type'] == 'EPHYS_1_images_G_3uL_reward'
g_sessions = behavior_sessions[mask]
mask = (behavior_sessions['age_in_days'] > 150) & (behavior_sessions['age_in_days'] <= 170)

g_sessions = g_sessions[mask]

# behavior_sessions['session_type'].unique()
display(g_sessions[['sex','genotype']].value_counts())

C:\Users\aeltona\AppData\Local\Temp\ipykernel_31096\2805055103.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  g_sessions = g_sessions[mask]


sex  genotype                                     
F    Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt    6
     wt/wt                                            6
M    Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt    5
     Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt    5
F    Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt    2
M    wt/wt                                            2
dtype: int64

In [ ]:
b

In [8]:
display(g_sessions[g_sessions['sex']=='F'])
# g_sessions['mouse_id'].value_counts()

,equipment_name,genotype,mouse_id,sex,age_in_days,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ecephys_session_id,date_of_acquisition,session_type,image_set,file_id
behavior_session_id,,,,,,,,,,,,,,
1048009327,NP.0,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,524925,F,165,57,0,51,0,1047977240,2020-09-02 15:15:03.733000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1056528863,NP.1,wt/wt,531237,F,165,50,0,43,0,1056495334,2020-10-14 15:11:39.817000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1061278841,NP.0,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,536213,F,156,43,0,34,0,1061238668,2020-11-04 14:22:22.327000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1077733165,NP.0,wt/wt,548715,F,156,32,0,19,0,1077712208,2021-01-20 14:17:46.814000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1081118392,NP.0,wt/wt,548720,F,170,30,0,23,0,1081079981,2021-02-03 13:49:34.800000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1079049704,NP.1,wt/wt,548717,F,163,25,0,16,0,1079018622,2021-01-27 13:58:36.712000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1081125370,NP.1,wt/wt,548721,F,170,29,0,23,0,1081090969,2021-02-03 14:22:10.165000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1087741158,NP.1,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,553964,F,164,28,0,18,0,1087723305,2021-03-03 14:48:42.566000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1086220230,NP.1,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,554013,F,156,27,0,18,0,1086200042,2021-02-24 14:33:38.933000+00:00,EPHYS_1_images_G_3uL_reward,G,-999


In [70]:
# ecephys_sessions_list = [1056495334, 1061238668, 1124507277]
# name_list = ["wt", "vip", "sst"]	
# common_values = behavior_sessions['ecephys_session_id'].isin(ecephys_sessions_list)
# behavior_sessions[common_values]

In [11]:
ecephys_sessions_list = [1056495334, 1061238668, 1124507277]
name_list = ["wt", "vip", "sst"]	
common_values = g_sessions['ecephys_session_id'].isin(ecephys_sessions_list)
unique_sessions = g_sessions[common_values]
unique_sessions

# common_values = ecephys_sessions.index.values.isin(ecephys_sessions_list)

,equipment_name,genotype,mouse_id,sex,age_in_days,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ecephys_session_id,date_of_acquisition,session_type,image_set,file_id
behavior_session_id,,,,,,,,,,,,,,
1056528863,NP.1,wt/wt,531237,F,165,50,0,43,0,1056495334,2020-10-14 15:11:39.817000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1061278841,NP.0,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,536213,F,156,43,0,34,0,1061238668,2020-11-04 14:22:22.327000+00:00,EPHYS_1_images_G_3uL_reward,G,-999
1124536942,NP.0,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,577287,F,159,48,0,42,1,1124507277,2021-08-26 14:32:42.128000+00:00,EPHYS_1_images_G_3uL_reward,G,-999


In [13]:
for session_id, name in zip(ecephys_sessions_list, name_list):
    print("Loading session",session_id, name)
    globals()[f'session_{name}'] = cache.get_ecephys_session(ecephys_session_id=session_id)
    # print(f"Session {session_id} has {len(session.trials)} trials")
    # print(f"Session {session_id} has {len(session.stimulus_presentations)} stimulus presentations")
    # print(f"Session {session_id} has {len(session.stimulus_templates)} stimulus templates")
    # print(f"Session {session_id} has {len(session.stimulus_conditions)} stimulus conditions")
    # print(f"Session {session_id} has {len(session.stimulus_timestamps)} stimulus timestamps")
    # print(f"Session {session_id} has {len(session.stimulus_names)} stimulus names")
    # print(f"Session {session_id} has {len(session.stimulus_metadata)} stimulus metadata")
    # print(f"Session {session_id} has {len(session.stimulus_block_table)} stimulus block table

Loading session 1056495334 wt


ecephys_session_1056495334.nwb: 100%|██████████| 3.43G/3.43G [27:24<00:00, 2.09MMB/s]    
c:\myprograms\Anaconda\envs\behavis\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Loading session 1061238668 vip


ecephys_session_1061238668.nwb:   9%|▉         | 298M/3.25G [02:36<26:08, 1.88MMB/s]     

KeyboardInterrupt: 

In [19]:
# dir(session_wt)
# stimulus_presentations = session.stimulus_presentations
# stimulus_presentations.columns

In [18]:
# common_values = units['ecephys_session_id'].isin(ecephys_sessions_list)
# unique_units = units[common_values]
# unique_units['structure_acronym'].value_counts()

In [69]:
# mask = sliced_channels['unit_count'] > 0
# sliced_channels[mask]['unit_count'].value_counts()


# Slicing data by stimulus timestamps

In [21]:
stimulus_presentations = session_wt.stimulus_presentations
stimulus_presentations.columns

Index(['stimulus_block', 'image_name', 'duration', 'start_time', 'end_time',
       'start_frame', 'end_frame', 'is_change', 'is_image_novel', 'omitted',
       'flashes_since_change', 'trials_id', 'position_y', 'rewarded',
       'position_x', 'temporal_frequency', 'stimulus_index', 'color',
       'spatial_frequency', 'active', 'is_sham_change', 'contrast',
       'orientation', 'stimulus_name'],
      dtype='object')

In [28]:
# mask = (stimulus_presentations['stimulus_block'] == 0) | (stimulus_presentations['stimulus_block'] == 1)
mask = (stimulus_presentations['stimulus_block'] == 0) 
# mask = (stimulus_presentations['stimulus_block'] == 1)
nat_imgs = stimulus_presentations[mask]
nat_imgs["omitted"].value_counts()    

False    4621
True      182
Name: omitted, dtype: Int64

In [43]:
idx.to_numpy()

array([  19,   54,   78,  107,  201,  240,  251,  257,  265,  267,  327,
        408,  417,  420,  422,  430,  450,  471,  479,  496,  514,  565,
        583,  596,  598,  610,  636,  703,  761,  763,  842,  889,  903,
        907,  914,  944,  996,  998, 1009, 1043, 1045, 1074, 1097, 1120,
       1129, 1149, 1158, 1171, 1181, 1262, 1293, 1329, 1337, 1371, 1391,
       1423, 1428, 1467, 1528, 1538, 1560, 1573, 1587, 1590, 1611, 1679,
       1692, 1700, 1715, 1719, 1724, 1787, 1807, 1839, 1845, 1850, 1928,
       1961, 1994, 2025, 2037, 2136, 2144, 2149, 2213, 2294, 2344, 2401,
       2442, 2468, 2533, 2535, 2562, 2569, 2609, 2657, 2724, 2733, 2759,
       2769, 2775, 2817, 2866, 2885, 2947, 2953, 3060, 3101, 3105, 3118,
       3127, 3151, 3169, 3204, 3243, 3271, 3300, 3327, 3336, 3340, 3347,
       3377, 3393, 3402, 3429, 3534, 3548, 3564, 3605, 3609, 3616, 3619,
       3631, 3640, 3649, 3679, 3751, 3757, 3791, 3823, 3826, 3875, 3879,
       3898, 3954, 3957, 3966, 3993, 3995, 4022, 40

In [52]:
mask = nat_imgs["omitted"]==True
idx = nat_imgs[mask].index.to_numpy()
size = len(idx) *5
pos_idxs = np.concatenate([idx+i for i in range(1,5)])
neg_idxs = np.concatenate([idx-i for i in range(5)])
all_idxs = np.concatenate([pos_idxs, neg_idxs])
all_idxs.sort()
nat_imgs.iloc[all_idxs].head(20)

,stimulus_block,image_name,duration,start_time,end_time,start_frame,end_frame,is_change,is_image_novel,omitted,...,position_x,temporal_frequency,stimulus_index,color,spatial_frequency,active,is_sham_change,contrast,orientation,stimulus_name
stimulus_presentations_id,,,,,,,,,,,,,,,,,,,,,
15,0,im036_r,0.250196,36.858354,37.108550,735,750,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
16,0,im115_r,0.250214,37.608941,37.859155,780,795,True,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
17,0,im115_r,0.250214,38.359584,38.609799,825,840,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
18,0,im115_r,0.250211,39.110220,39.360430,870,885,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
19,0,omitted,NaN,39.860850,40.111060,915,930,False,<NA>,True,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
20,0,im115_r,0.250208,40.611479,40.861687,960,975,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
21,0,im115_r,0.266889,41.362103,41.628992,1005,1021,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
22,0,im115_r,0.250214,42.112740,42.362954,1050,1065,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019
23,0,im115_r,0.250198,42.863367,43.113565,1095,1110,False,False,False,...,NaN,NaN,-99,NaN,NaN,True,False,NaN,NaN,Natural_Images_Lum_Matched_set_ophys_G_2019


In [66]:
all_idxs = []
for i in idx:
    omitted_stim = nat_imgs.iloc[i-1]['image_name']
    all_idxs.append(i)
    all_idxs.append(i+1)
    
    count = 1
    stop = 0
    while stop == 0:
        if nat_imgs.iloc[i-count]['image_name'] == omitted_stim:
            all_idxs.append(i-count)
        else:
            stop = 1           
        count += 1   
all_idxs = list(set(all_idxs))
len(all_idxs)      
    # all_idxs.append(i+1)

1847

In [68]:
all_idxs.sort()
omitted_trials = nat_imgs.iloc[all_idxs]

In [ ]:
region_dict = {'cortex' : ['VISp', 'VISl', 'VISrl', 'VISam', 'VISpm', 'VIS', 'VISal','VISmma','VISmmp','VISli'],
             'thalamus' : ['LGd','LD', 'LP', 'VPM', 'TH', 'MGm','MGv','MGd','PO','LGv','VL',
              'VPL','POL','Eth','PoT','PP','PIL','IntG','IGL','SGN','VPL','PF','RT'],
             'hippocampus' : ['CA1', 'CA2','CA3', 'DG', 'SUB', 'POST','PRE','ProS','HPF'],
             'midbrain': ['MB','SCig','SCiw','SCsg','SCzo','PPT','APN','NOT','MRN','OP','LT','RPF','CP']}

In [ ]:
# probes

In [73]:
mask = probes['ecephys_session_id']==1056495334
probes[mask]

,ecephys_session_id,name,sampling_rate,lfp_sampling_rate,phase,has_lfp_data,unit_count,channel_count,structure_acronyms,file_id
ecephys_probe_id,,,,,,,,,,
1056630213,1056495334,probeA,29999.995365,2499.999614,1.0,True,375,384,"['VISam', 'grey', 'root']",128
1056630214,1056495334,probeB,30000.201305,2500.016775,1.0,True,399,384,"['VISpm', 'grey', 'root']",129
1056630215,1056495334,probeC,30000.068595,2500.005716,1.0,True,206,384,"['VISp', 'grey', 'root']",130
1056630217,1056495334,probeD,29999.938593,2499.994883,1.0,True,613,384,"['VISl', 'grey', 'root']",131
1056630218,1056495334,probeE,30000.096824,2500.008069,1.0,True,570,384,"['VISal', 'grey', 'root']",132
1056630219,1056495334,probeF,30000.113075,2500.009423,1.0,True,581,384,"['VISrl', 'grey', 'root']",133


In [ ]:
session_wt.get_lfp(1056630213)


probe_probeA_lfp.nwb:   8%|▊         | 384M/4.74G [03:36<34:36, 2.10MMB/s]     